#### Import modules

In [11]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [13]:
width, height = 200, 150
img = np.zeros((height, width, 3), dtype=np.uint8)

# Set RGB values
red, green, blue = 255, 255, 255

img[:,:,0] = red
img[:,:,1] = green
img[:,:,2] = blue

In [30]:
def rgb_to_rgb565(rgb_img):
    """
    Convert RGB image (height, width, 3)[uint8] into RGB565 (height, width)[uint16]
    """
    h, w, _ = rgb_img.shape
    rgb565 = np.zeros((height, width), dtype=np.uint16)
    
    for y in range(height):
        for x in range(width):
            r, g, b = rgb_img[y, x]
            new_r = r >> 3
            new_g = g >> 2
            new_b = b >> 3
            color = (new_r << 11) | ((new_g) << 5) | new_b
            rgb565[y, x] = color
    return rgb565

def img_to_bytes(img):
    """
    Convert RGB image into RGB565 bytes
    """
    rgb565_img = rgb_to_rgb565(img)
    img_bytes = rgb565_img.flatten().tobytes()
    return img_bytes

In [48]:
import paho.mqtt.client as mqtt

In [50]:
mqtt_broker = '192.168.1.101'
mqtt_username = 'testUser'
mqtt_password = 'Abc_072179'
topic = 'test/esp32'

In [58]:
class MqttPublisher:
    def __init__(self, broker_addr='localhost', port=1883):
        self.mqtt_client = mqtt.Client()
        self.broker_addr = broker_addr
        self.port = port
        self.topics = [] # 'test/esp32'
        
        self._credential = None # Encoded credentials
    
    def add_topic(self, topic):
        self.topics.append(topic)
    
    def remove_topic(self, topic):
        self.topics.remove(topic)
    
    def set_credential(self, username, pw):
        self._credential((username, pw))
    
    def connect(self):
        if self._credentials:
            username, pw = self._credentials
            self.mqtt_client.username_pw_set(username, pw)
        self.mqtt_client.connect(self.broker_addr, self.port)
        
    def publish(self, payload):
        if not self.mqtt_client.is_connected():
            print('Client is not connected to the broker yet. Hint: Call start() to try connecting again')
            return
        for topic in self.topics:
            self.mqtt_client.publish(topic, payload)

In [60]:
class ImageSender:
    TOPIC = 'test/esp32'
    BROKER_ADDR = '192.168.1.101'
    PORT = 1883
    USERNAME = 'testUser'
    PASSWORD = 'Abc_072179'
    
    def __init__(self):
        self.mqtt_publisher = MqttPublisher(broker_addr=BROKER_ADDR, port=PORT)
        self.mqtt_publisher.set_credential(USERNAME, PASSWORD)
        self.add_topic(TOPIC)
        self.mqtt_publisher.connect()
    
    def send(self, img):
        img_bytes = img_to_bytes(img)
        for i in range(0, len(img_bytes), chunk_size):
            data_chunk = img_bytes[i:i+chunk_size]
            self.mqtt_publish.publish(data_chunk)

In [ ]:
# sender = ImageSender()
# sender.send(img)